# FM model base + USD + WTI

In [125]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

# data

In [126]:
# country를 index로 바꾼 pickle 파일을 불러오기.
import pickle
with open('../processed/country_to_idx.pkl', 'rb') as f:
    country_to_idx = pickle.load(f)

with open('../processed/idx_to_country.pkl', 'rb') as f:
    idx_to_country = pickle.load(f)

In [ ]:
# country를 index로 바꾼 pickle 파일을 불러오기.
with open('../processed/new_exp_3years.pkl', 'rb') as f:
    new_exp_3years = pickle.load(f)


In [127]:
merge_all = pd.read_csv('../processed/export_diff_label.csv')
merge_all

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128388,2022,Zimbabwe,90.0,299.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0
128389,2022,Zimbabwe,94.0,21.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0
128390,2022,Zimbabwe,95.0,2.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0
128391,2022,Zimbabwe,96.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0


In [128]:
# country_id를 고유번호로 바꾸기
merge_all['country'] = merge_all['country'].map(country_to_idx)
merge_all

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
0,2012,0,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
1,2012,0,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
2,2012,0,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
3,2012,0,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
4,2012,0,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128388,2022,202,90.0,299.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0
128389,2022,202,94.0,21.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0
128390,2022,202,95.0,2.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0
128391,2022,202,96.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000,0


##### -

In [129]:
with open('../processed/export_GDP_pop_fx_wti.pkl', 'rb') as f:
    data = pd.read_pickle(f)

In [130]:
data

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
1,2012,Afghanistan,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
2,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
3,2012,Afghanistan,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
4,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128388,2022,Zimbabwe,90.0,299.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128389,2022,Zimbabwe,94.0,21.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128390,2022,Zimbabwe,95.0,2.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128391,2022,Zimbabwe,96.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000


In [131]:
# country_id를 고유번호로 바꾸기
data['country'] = data['country'].map(country_to_idx)
data

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI
0,2012,0,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
1,2012,0,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
2,2012,0,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
3,2012,0,8.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
4,2012,0,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128388,2022,202,90.0,299.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128389,2022,202,94.0,21.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128390,2022,202,95.0,2.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000
128391,2022,202,96.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.150000


In [132]:
# 2012년도는 growth를 보기 힘드므로 제외
data = data[data['year'] != 2012]
data.rename(columns={'품목코드':'item_id'}, inplace=True)
data.head(3)

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38787/3420230340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'품목코드':'item_id'}, inplace=True)


,year,country,item_id,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI
11026,2013,0,10.0,1.0,2.056449e+07,4.862752e-08,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075
11027,2013,0,16.0,13.0,2.056449e+07,6.321578e-07,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075
11028,2013,0,19.0,37.0,2.056449e+07,1.799218e-06,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075


In [133]:
merge_all.head(3)

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
0,2012,0,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
1,2012,0,4.0,0.0,20203572.96,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
2,2012,0,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0


In [137]:
merge_all.rename(columns={'country_id':'country'}, inplace=True)
merge_all.rename(columns={'품목코드':'item_id'}, inplace=True)

In [138]:
ratings = pd.merge(data, merge_all[['country','year','item_id','label']], how='left', on=['country','year','item_id'])
ratings

,year,country,item_id,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
0,2013,0,10.0,1.0,2.056449e+07,4.862752e-08,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075,1
1,2013,0,16.0,13.0,2.056449e+07,6.321578e-07,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075,1
2,2013,0,19.0,37.0,2.056449e+07,1.799218e-06,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075,0
3,2013,0,20.0,1.0,2.056449e+07,4.862752e-08,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075,1
4,2013,0,21.0,142.0,2.056449e+07,6.905108e-06,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117362,2022,202,90.0,299.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.1500,0
117363,2022,202,94.0,21.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.1500,0
117364,2022,202,95.0,2.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.1500,0
117365,2022,202,96.0,0.0,NaN,NaN,NaN,NaN,NaN,1293.68,985.24,1359.46,191.73,88.1500,0


In [139]:
# USD만 냅두고 나머지 FX들은 제거하기
ratings=ratings[['year','country','item_id','GDP','preference','GDP_growth','population','pop_growth','USD','WTI','label']]
ratings.describe()

,year,country,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
count,117367.000000,117367.000000,117367.000000,1.053690e+05,1.053690e+05,105369.000000,1.053690e+05,105369.000000,117367.000000,117367.000000,117367.000000
mean,2017.559058,103.884175,50.803343,6.404606e+08,1.920689e-04,0.038022,5.480439e+07,0.011590,1146.622029,65.008431,0.363756
std,2.868819,57.699984,27.364460,2.209886e+09,3.325642e-03,0.295688,1.777078e+08,0.013928,61.226846,19.639027,0.481082
min,2013.000000,0.000000,1.000000,3.681166e+04,0.000000e+00,-0.999033,1.069400e+04,-0.041673,1053.300000,38.595833,0.000000
25%,2015.000000,55.000000,28.000000,1.898180e+07,3.418803e-08,-0.016736,4.026336e+06,0.002953,1101.080000,49.307500,0.000000
50%,2018.000000,102.000000,51.000000,7.893026e+07,1.385968e-06,0.037601,1.075468e+07,0.010646,1145.070000,64.544167,0.000000
75%,2020.000000,151.000000,73.000000,3.750000e+08,1.713603e-05,0.088785,3.801174e+07,0.019160,1166.510000,88.150000,1.000000
max,2022.000000,202.000000,99.000000,2.330000e+10,3.165603e-01,8.974569,1.412360e+09,0.225784,1293.680000,97.607500,1.000000


In [140]:
# 2012년도 USD로 나누기 > 정규화
data = ratings.iloc[0]['USD']
ratings['USD'] =ratings['USD'] / data

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38787/1193071157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['USD'] =ratings['USD'] / data


In [141]:
# WTI min-max 정규화
ratings['WTI'] = (ratings['WTI'] - ratings['WTI'].min()) / (ratings['WTI'].max() - ratings['WTI'].min())
# population min-max 정규화
ratings['population'] = (ratings['population'] - ratings['population'].min()) / (ratings['population'].max() - ratings['population'].min())
# GDP min-max 정규화
ratings['GDP'] = (ratings['GDP'] - ratings['GDP'].min()) / (ratings['GDP'].max() - ratings['GDP'].min())
ratings

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38787/4016649771.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['WTI'] = (ratings['WTI'] - ratings['WTI'].min()) / (ratings['WTI'].max() - ratings['WTI'].min())
/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38787/4016649771.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['population'] = (ratings['population'] - ratings['population'].min()) / (ratings['population'].max() - ratings['population'].min())
/var/folder

,year,country,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
0,2013,0,10.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.000000,1.000000,1
1,2013,0,16.0,0.000881,6.321578e-07,0.017864,0.022325,0.035276,1.000000,1.000000,1
2,2013,0,19.0,0.000881,1.799218e-06,0.017864,0.022325,0.035276,1.000000,1.000000,0
3,2013,0,20.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.000000,1.000000,1
4,2013,0,21.0,0.000881,6.905108e-06,0.017864,0.022325,0.035276,1.000000,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...
117362,2022,202,90.0,NaN,NaN,NaN,NaN,NaN,1.181475,0.839735,0
117363,2022,202,94.0,NaN,NaN,NaN,NaN,NaN,1.181475,0.839735,0
117364,2022,202,95.0,NaN,NaN,NaN,NaN,NaN,1.181475,0.839735,0
117365,2022,202,96.0,NaN,NaN,NaN,NaN,NaN,1.181475,0.839735,0


In [142]:
ratings.describe()

,year,country,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
count,117367.000000,117367.000000,117367.000000,105369.000000,1.053690e+05,105369.000000,105369.000000,105369.000000,117367.000000,117367.000000,117367.000000
mean,2017.559058,103.884175,50.803343,0.027486,1.920689e-04,0.038022,0.038796,0.011590,1.047172,0.447583,0.363756
std,2.868819,57.699984,27.364460,0.094845,3.325642e-03,0.295688,0.125824,0.013928,0.055916,0.332799,0.481082
min,2013.000000,0.000000,1.000000,0.000000,0.000000e+00,-0.999033,0.000000,-0.041673,0.961944,0.000000,0.000000
25%,2015.000000,55.000000,28.000000,0.000813,3.418803e-08,-0.016736,0.002843,0.002953,1.005580,0.181518,0.000000
50%,2018.000000,102.000000,51.000000,0.003386,1.385968e-06,0.037601,0.007607,0.010646,1.045755,0.439715,0.000000
75%,2020.000000,151.000000,73.000000,0.016093,1.713603e-05,0.088785,0.026906,0.019160,1.065335,0.839735,1.000000
max,2022.000000,202.000000,99.000000,1.000000,3.165603e-01,8.974569,1.000000,0.225784,1.181475,1.000000,1.000000


In [143]:
ratings.rename(columns={'country':'country_id'}, inplace=True)

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38787/783160813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings.rename(columns={'country':'country_id'}, inplace=True)


In [144]:
ratings

,year,country_id,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
0,2013,0,10.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.000000,1.000000,1
1,2013,0,16.0,0.000881,6.321578e-07,0.017864,0.022325,0.035276,1.000000,1.000000,1
2,2013,0,19.0,0.000881,1.799218e-06,0.017864,0.022325,0.035276,1.000000,1.000000,0
3,2013,0,20.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.000000,1.000000,1
4,2013,0,21.0,0.000881,6.905108e-06,0.017864,0.022325,0.035276,1.000000,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...
117362,2022,202,90.0,NaN,NaN,NaN,NaN,NaN,1.181475,0.839735,0
117363,2022,202,94.0,NaN,NaN,NaN,NaN,NaN,1.181475,0.839735,0
117364,2022,202,95.0,NaN,NaN,NaN,NaN,NaN,1.181475,0.839735,0
117365,2022,202,96.0,NaN,NaN,NaN,NaN,NaN,1.181475,0.839735,0


In [145]:
ratings = ratings.query('year < 2022')
ratings

,year,country_id,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
0,2013,0,10.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.000000,1.000000,1
1,2013,0,16.0,0.000881,6.321578e-07,0.017864,0.022325,0.035276,1.000000,1.000000,1
2,2013,0,19.0,0.000881,1.799218e-06,0.017864,0.022325,0.035276,1.000000,1.000000,0
3,2013,0,20.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.000000,1.000000,1
4,2013,0,21.0,0.000881,6.905108e-06,0.017864,0.022325,0.035276,1.000000,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...
105364,2021,202,87.0,0.001216,4.543334e-05,0.318998,0.011316,0.020668,1.045755,0.502048,1
105365,2021,202,90.0,0.001216,1.134952e-05,0.318998,0.011316,0.020668,1.045755,0.502048,0
105366,2021,202,94.0,0.001216,3.524696e-08,0.318998,0.011316,0.020668,1.045755,0.502048,1
105367,2021,202,95.0,0.001216,4.582105e-07,0.318998,0.011316,0.020668,1.045755,0.502048,0


In [146]:
ratings.isnull().sum()

year          0
country_id    0
item_id       0
GDP           0
preference    0
GDP_growth    0
population    0
pop_growth    0
USD           0
WTI           0
label         0
dtype: int64

## test > CF에서 가져온 것들

In [147]:
test_set = pd.read_csv('../processed/test12_18_new.csv')
test_set

,country_id,year,item_id,GDP_growth,pop_growth,preference
0,0,2012,1.0,NaN,NaN,8.676877e-07
1,0,2012,2.0,NaN,NaN,4.316267e-06
2,0,2012,4.0,NaN,NaN,1.464146e-06
3,0,2012,5.0,NaN,NaN,1.516132e-06
4,0,2012,6.0,NaN,NaN,1.241186e-06
...,...,...,...,...,...,...
69793,202,2018,89.0,0.942353,0.020411,1.069936e-05
69794,202,2018,91.0,0.942353,0.020411,7.333262e-07
69795,202,2018,92.0,0.942353,0.020411,6.773078e-07
69796,202,2018,97.0,0.942353,0.020411,7.280952e-10


In [148]:
test_set = test_set[test_set.year != 2012]
test_set

,country_id,year,item_id,GDP_growth,pop_growth,preference
56,0,2013,1.0,0.017864,0.035276,1.730195e-08
57,0,2013,2.0,0.017864,0.035276,1.115475e-07
58,0,2013,3.0,0.017864,0.035276,1.524764e-05
59,0,2013,4.0,0.017864,0.035276,3.985107e-07
60,0,2013,5.0,0.017864,0.035276,1.581240e-07
...,...,...,...,...,...,...
69793,202,2018,89.0,0.942353,0.020411,1.069936e-05
69794,202,2018,91.0,0.942353,0.020411,7.333262e-07
69795,202,2018,92.0,0.942353,0.020411,6.773078e-07
69796,202,2018,97.0,0.942353,0.020411,7.280952e-10


In [149]:
# data 합치기
ratings = pd.concat([ratings, test_set])
ratings.reset_index(drop=True, inplace=True)
ratings

,year,country_id,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
0,2013,0,10.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.0,1.0,1.0
1,2013,0,16.0,0.000881,6.321578e-07,0.017864,0.022325,0.035276,1.0,1.0,1.0
2,2013,0,19.0,0.000881,1.799218e-06,0.017864,0.022325,0.035276,1.0,1.0,0.0
3,2013,0,20.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.0,1.0,1.0
4,2013,0,21.0,0.000881,6.905108e-06,0.017864,0.022325,0.035276,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
164838,2018,202,89.0,NaN,1.069936e-05,0.942353,NaN,0.020411,NaN,NaN,NaN
164839,2018,202,91.0,NaN,7.333262e-07,0.942353,NaN,0.020411,NaN,NaN,NaN
164840,2018,202,92.0,NaN,6.773078e-07,0.942353,NaN,0.020411,NaN,NaN,NaN
164841,2018,202,97.0,NaN,7.280952e-10,0.942353,NaN,0.020411,NaN,NaN,NaN


In [150]:
from tqdm import tqdm

# USD/WTI 채워넣기
for i in tqdm(range(2013,2019)):
    ratings.loc[(ratings['year']==i) & (ratings['USD'].isnull()), 'USD'] = ratings[ratings['year']==i]['USD'].unique()[0]
    ratings.loc[(ratings['year']==i) & (ratings['WTI'].isnull()), 'WTI'] = ratings[ratings['year']==i]['WTI'].unique()[0]
    

100%|██████████| 6/6 [00:00<00:00, 273.89it/s]


In [151]:
#gdp 채워넣기
for i in tqdm(range(2013,2019)):
    for j in range(len(ratings['country_id'].unique())):
        if i != 2013 and j in [133,134]:
            pass
        else:
            GDP_value = list(ratings.loc[(ratings['year']== i) & (ratings['country_id']==j) & (ratings['GDP'].notnull())].GDP)[0]
            ratings.loc[(ratings['year']== i) & (ratings['country_id']==j) & (ratings['GDP'].isnull()), 'GDP'] = GDP_value
            

100%|██████████| 6/6 [00:00<00:00,  7.51it/s]


In [152]:
# 년도와, country_id와가 일치하면 population 채워 넣기
for i in tqdm(range(2013,2019)):
    for j in range(len(ratings['country_id'].unique())):
        if i != 2013 and j in [133,134]:
            pass
        else:
            population = list(ratings.loc[(ratings['year']== i) & (ratings['country_id']==j) & (ratings['population'].notnull())].population)[0]
            ratings.loc[(ratings['year']== i) & (ratings['country_id']==j) & (ratings['population'].isnull()), 'population'] = population

100%|██████████| 6/6 [00:00<00:00,  7.21it/s]


In [153]:
ratings

,year,country_id,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
0,2013,0,10.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
1,2013,0,16.0,0.000881,6.321578e-07,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
2,2013,0,19.0,0.000881,1.799218e-06,0.017864,0.022325,0.035276,1.00000,1.000000,0.0
3,2013,0,20.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
4,2013,0,21.0,0.000881,6.905108e-06,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...
164838,2018,202,89.0,0.001464,1.069936e-05,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
164839,2018,202,91.0,0.001464,7.333262e-07,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
164840,2018,202,92.0,0.001464,6.773078e-07,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
164841,2018,202,97.0,0.001464,7.280952e-10,0.942353,0.010650,0.020411,1.00558,0.439715,NaN


In [154]:
ratings.isnull().sum()


year              0
country_id        0
item_id           0
GDP               0
preference        0
GDP_growth        0
population        0
pop_growth        0
USD               0
WTI               0
label         59474
dtype: int64

In [155]:
ratings

,year,country_id,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
0,2013,0,10.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
1,2013,0,16.0,0.000881,6.321578e-07,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
2,2013,0,19.0,0.000881,1.799218e-06,0.017864,0.022325,0.035276,1.00000,1.000000,0.0
3,2013,0,20.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
4,2013,0,21.0,0.000881,6.905108e-06,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...
164838,2018,202,89.0,0.001464,1.069936e-05,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
164839,2018,202,91.0,0.001464,7.333262e-07,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
164840,2018,202,92.0,0.001464,6.773078e-07,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
164841,2018,202,97.0,0.001464,7.280952e-10,0.942353,0.010650,0.020411,1.00558,0.439715,NaN


## train, test 나누기

In [156]:
train = ratings[ratings['label'].notnull()]
train.reset_index(drop=True, inplace=True)
train

,year,country_id,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
0,2013,0,10.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.000000,1.000000,1.0
1,2013,0,16.0,0.000881,6.321578e-07,0.017864,0.022325,0.035276,1.000000,1.000000,1.0
2,2013,0,19.0,0.000881,1.799218e-06,0.017864,0.022325,0.035276,1.000000,1.000000,0.0
3,2013,0,20.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.000000,1.000000,1.0
4,2013,0,21.0,0.000881,6.905108e-06,0.017864,0.022325,0.035276,1.000000,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...
105364,2021,202,87.0,0.001216,4.543334e-05,0.318998,0.011316,0.020668,1.045755,0.502048,1.0
105365,2021,202,90.0,0.001216,1.134952e-05,0.318998,0.011316,0.020668,1.045755,0.502048,0.0
105366,2021,202,94.0,0.001216,3.524696e-08,0.318998,0.011316,0.020668,1.045755,0.502048,1.0
105367,2021,202,95.0,0.001216,4.582105e-07,0.318998,0.011316,0.020668,1.045755,0.502048,0.0


In [157]:
# rating에서 diff가 null이 아닌 것들만 추출
test = ratings[ratings['label'].isnull()]
test.reset_index(drop=True, inplace=True)
test

,year,country_id,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
0,2013,0,1.0,0.000881,1.730195e-08,0.017864,0.022325,0.035276,1.00000,1.000000,NaN
1,2013,0,2.0,0.000881,1.115475e-07,0.017864,0.022325,0.035276,1.00000,1.000000,NaN
2,2013,0,3.0,0.000881,1.524764e-05,0.017864,0.022325,0.035276,1.00000,1.000000,NaN
3,2013,0,4.0,0.000881,3.985107e-07,0.017864,0.022325,0.035276,1.00000,1.000000,NaN
4,2013,0,5.0,0.000881,1.581240e-07,0.017864,0.022325,0.035276,1.00000,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
59469,2018,202,89.0,0.001464,1.069936e-05,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
59470,2018,202,91.0,0.001464,7.333262e-07,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
59471,2018,202,92.0,0.001464,6.773078e-07,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
59472,2018,202,97.0,0.001464,7.280952e-10,0.942353,0.010650,0.020411,1.00558,0.439715,NaN


In [158]:
# data 합치기 > encoding을 위해서
ratings = pd.concat([train, test])
ratings.reset_index(drop=True, inplace=True)
ratings

,year,country_id,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,label
0,2013,0,10.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
1,2013,0,16.0,0.000881,6.321578e-07,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
2,2013,0,19.0,0.000881,1.799218e-06,0.017864,0.022325,0.035276,1.00000,1.000000,0.0
3,2013,0,20.0,0.000881,4.862752e-08,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
4,2013,0,21.0,0.000881,6.905108e-06,0.017864,0.022325,0.035276,1.00000,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...
164838,2018,202,89.0,0.001464,1.069936e-05,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
164839,2018,202,91.0,0.001464,7.333262e-07,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
164840,2018,202,92.0,0.001464,6.773078e-07,0.942353,0.010650,0.020411,1.00558,0.439715,NaN
164841,2018,202,97.0,0.001464,7.280952e-10,0.942353,0.010650,0.020411,1.00558,0.439715,NaN


In [159]:
ratings.isnull().sum()

year              0
country_id        0
item_id           0
GDP               0
preference        0
GDP_growth        0
population        0
pop_growth        0
USD               0
WTI               0
label         59474
dtype: int64

# encoding

In [160]:
# year encoding
year_dict = {}
for i in set(ratings['year']):
    year_dict[i] = len(year_dict)
n_year = len(year_dict)                  

# Country encoding
country_dict = {}
start_point = n_year                     
for i in set(ratings['country_id']):
    country_dict[i] = len(country_dict)
n_country = len(country_dict)            
start_point += n_country                 

# Item encoding
item_dict = {}
start_point = n_country                  
for i in set(ratings['item_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)                  
start_point += n_item       

# GDP
gdp_index = start_point
start_point += 1          

# preference encoding
pre_index = start_point
start_point += 1

# GDP_growth encoding   
gdp_growth_index = start_point
start_point += 1

# population encoding   
population_index = start_point
start_point += 1

# pop_growth encoding
pop_growth_index = start_point
start_point += 1

# 환율
USD_index = start_point
start_point += 1

# 유가
WTI_index = start_point
start_point += 1

num_x = start_point                    

In [161]:
x = ratings

In [162]:
# Generate X data
data = []
y = []
for i in range(len(x)):
    case = x.iloc[i]
    x_index = []
    x_value = []
    x_index.append(year_dict[case['year']])        # year id one-hot encoding
    x_value.append(1.)                             # 있으면 1
    
    x_index.append(country_dict[case['country_id']])  # country id one-hot encoding
    x_value.append(1.)
    
    x_index.append(item_dict[case['item_id']])     # item id one-hot encoding
    x_value.append(1.)
    
    x_index.append(gdp_index)                       # GDP encoding  
    x_value.append(case['GDP'])
    
    x_index.append(pre_index)                   # preference  encoding
    x_value.append(case['preference'])
     
    x_index.append(gdp_growth_index)              # GDP_growth encoding
    x_value.append(case['GDP_growth'])
    
    x_index.append(population_index)            # population encoding
    x_value.append(case['population'])
    
    x_index.append(pop_growth_index)        # pop_growth encoding
    x_value.append(case['pop_growth'])
    
    x_index.append(USD_index)           # USD encoding
    x_value.append(case['USD'])
    
    x_index.append(WTI_index)              # WTI encoding
    x_value.append(case['WTI'])
    
    data.append([x_index, x_value])                
    y.append(case['label'])                     
      
    if (i % 30000) == 0:                # 진행상황 확인
        print('Encoding ', i, ' cases...')

Encoding  0  cases...
Encoding  30000  cases...
Encoding  60000  cases...
Encoding  90000  cases...
Encoding  120000  cases...
Encoding  150000  cases...


In [163]:
# split 할 부분 찾기
idx = ratings[ratings['label'].notnull()].index[-1]
idx

105368

In [164]:
trainset = data[:idx+1]
testset = data[idx+1:]
y = y[:idx+1]

In [165]:
# 갯수확인
print(len(trainset))
print(len(testset))
print(len(y))

105369
59474
105369


# model

In [166]:
class FactorizationMachine():
    def __init__(self, N, K, data, y, test_data, lr, l2_reg, l2_lambda, epoch, early_stop_window, train_ratio=0.75):
        """
        :param k: number of latent vector
        :param lr: learning rate
        :param l2_reg: bool parameter for L2 regularization
        :param l2_lambda: lambda of L2 regularization
        :param epoch: training epoch
        """
        self._K = K
        self._N = N
        self._data = data
        self._y = y
        self._test_data = test_data
        self._lr = lr
        self._l2_reg = l2_reg
        self._l2_lambda = l2_lambda
        self._epoch = epoch
        self._early_stop_window = early_stop_window
        self._valid_loss_list = []
        # Train/Valid/Test 분리
        cutoff = int(train_ratio * len(data))
        self.train_x = data[:cutoff]
        self.valid_x = data[cutoff:]
        self.train_y = y[:cutoff]
        self.valid_y = y[cutoff:]
        self.test_x = test_data

    def _load_dataset(self):
        # init FM vectors
        self._init_vectors()
        print("Finish init FM vectors.")

    def _init_vectors(self):
        # w 초기화
        self.w = np.random.normal(scale=1./self._N, size=(self._N))          # 사이즈는 변수의 수 만큼
        # v 초기화
        self.v = np.random.normal(scale=1./self._K, size=(self._N, self._K))  # 사이즈는 (변수의 수 X K)
        self.train_x_data = []
        self.train_y_data = np.zeros((len(self.train_x)))
        self.valid_x_data = []
        self.valid_y_data = np.zeros((len(self.valid_x)))

        self.train_y_data = np.array(self.train_y)
        for n, row in enumerate(self.train_x):
            self.train_x_data.append([np.array(row[0]), np.array(row[1])])
        
        self.valid_y_data = np.array(self.valid_y)
        for n, row in enumerate(self.valid_x):
            self.valid_x_data.append([np.array(row[0]), np.array(row[1])])

    def train(self):
        """
        Train FM model by Gradient Descent with L2 regularization
        """
        self._load_dataset()
        for epoch_num in range(1, self._epoch):
            train_y_hat = self.predict(data=self.train_x_data)
            valid_y_hat = self.predict(data=self.valid_x_data)
            train_loss = self._get_loss(y_data=self.train_y_data, y_hat=train_y_hat)
            valid_loss = self._get_loss(y_data=self.valid_y_data, y_hat=valid_y_hat)
            train_auc = roc_auc_score(self.train_y_data, train_y_hat)
            valid_auc = roc_auc_score(self.valid_y_data, valid_y_hat)
            self._print_learning_info(epoch=epoch_num, train_loss=train_loss, valid_loss=valid_loss,
                                      train_auc=train_auc, valid_auc=valid_auc)
            if self._check_early_stop(valid_loss=valid_loss):
                print("Early stop at epoch:", epoch_num)
                return 0

            self._stochastic_gradient_descent(self.train_x_data, self.train_y_data)

    def predict(self, data):
        """
        Implementation of FM model's equation on O(kmd)

        -----
        Numpy array shape : (n, [index of md], [value of md])
        md : none-zero feature
        """  
        num_data = len(data)
        scores = np.zeros(num_data)
        for n in range(num_data):
            feat_idx = data[n][0]
            val = data[n][1]

            # linear feature score
            linear_feature_score = np.sum(self.w[feat_idx] * val)

            # factorized feature score
            vx = self.v[feat_idx] * (val.reshape(-1, 1))
            cross_sum = np.sum(vx, axis=0)
            square_sum = np.sum(vx * vx, axis=0)
            cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

            # Model's equation
            scores[n] = linear_feature_score + cross_feature_score

        # Sigmoid transformation for binary classification
        scores = 1.0 / (1.0 + np.exp(-scores))
        return scores

    def inference(self, data):
        
        self.test_x_data = []
        for n, row in enumerate(self.test_x):
            self.test_x_data.append([np.array(row[0]), np.array(row[1])])

        num_data = len(data)
        scores = np.zeros(num_data)
        for n in range(num_data):
            feat_idx = data[n][0]
            val = np.array(data[n][1])

            # linear feature score
            linear_feature_score = np.sum(self.w[feat_idx] * val)

            # factorized feature score
            vx = self.v[feat_idx] * (val.reshape(-1, 1))
            cross_sum = np.sum(vx, axis=0)
            square_sum = np.sum(vx * vx, axis=0)
            cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

            # Model's equation
            scores[n] = linear_feature_score + cross_feature_score

        # Sigmoid transformation for binary classification
        scores = 1.0 / (1.0 + np.exp(-scores))
        print(scores)
        return scores

    def _get_loss(self, y_data, y_hat):
        """
        Calculate loss with L2 regularization (two type of coeficient - w,v)
        """
        l2_norm = 0
        if self._l2_reg:
            w_norm = np.sqrt(np.sum(np.square(self.w)))
            v_norm = np.sqrt(np.sum(np.square(self.v)))
            l2_norm = self._l2_lambda * (w_norm + v_norm)
        return -1 * np.sum( (y_data * np.log(y_hat)) + ((1 - y_data) * np.log(1 - y_hat)) ) + l2_norm

    def _check_early_stop(self, valid_loss):
        self._valid_loss_list.append(valid_loss)
        if len(self._valid_loss_list) > 5:
            prev_loss = self._valid_loss_list[len(self._valid_loss_list) - self._early_stop_window]
            curr_loss = valid_loss
            if prev_loss < curr_loss:
                return True
        return False

    def _print_learning_info(self, epoch, train_loss, valid_loss, train_auc, valid_auc):
        print("epoch:", epoch, "||", "train_loss:", train_loss, "||", "valid_loss:", valid_loss,
              "||", "Train AUC:", train_auc, "||", "Test AUC:", valid_auc)

    def _stochastic_gradient_descent(self, x_data, y_data):
        """
        Update each coefs (w, v) by Gradient Descent
        """
        for data, y in zip(x_data, y_data):
            feat_idx = data[0]
            val = data[1]
            vx = self.v[feat_idx] * (val.reshape(-1, 1))

            # linear feature score
            linear_feature_score = np.sum(self.w[feat_idx] * val)

            # factorized feature score
            vx = self.v[feat_idx] * (val.reshape(-1, 1))
            cross_sum = np.sum(vx, axis=0)
            square_sum = np.sum(vx * vx, axis=0)
            cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

            # Model's equation
            score = linear_feature_score + cross_feature_score
            y_hat = 1.0 / (1.0 + np.exp(-score))
            cost = y_hat - y

            if self._l2_reg:
                self.w[feat_idx] = self.w[feat_idx] - cost * self._lr * (val + self._l2_lambda * self.w[feat_idx])
                self.v[feat_idx] = self.v[feat_idx] - cost * self._lr * ((sum(vx) * (val.reshape(-1, 1)) - (vx * (val.reshape(-1, 1)))) + self._l2_lambda * self.v[feat_idx])
            else:
                self.w[feat_idx] = self.w[feat_idx] - cost * self._lr * val
                self.v[feat_idx] = self.v[feat_idx] - cost * self._lr * (sum(vx) * (val.reshape(-1, 1)) - (vx * (val.reshape(-1, 1))))

In [167]:
if __name__ == "__main__":
    fm = FactorizationMachine(N = num_x,
                              K=4,
                              data = trainset,
                              y = y,
                              test_data = testset,
                              lr=0.001,
                              l2_reg=True,
                              l2_lambda=0.0001,
                              epoch=200,
                              early_stop_window=3)

    fm.train()
    predict_proba = fm.inference(data=testset)

Finish init FM vectors.
epoch: 1 || train_loss: 56179.89654747285 || valid_loss: 18544.39516652058 || Train AUC: 0.5039429397474304 || Test AUC: 0.5176476427951422
epoch: 2 || train_loss: 52814.18928577141 || valid_loss: 18119.101075189068 || Train AUC: 0.5568492805862715 || Test AUC: 0.5665802555981098
epoch: 3 || train_loss: 52000.010205127684 || valid_loss: 17921.330228626248 || Train AUC: 0.6006087728190997 || Test AUC: 0.5968108277496607
epoch: 4 || train_loss: 51537.16259432322 || valid_loss: 17767.896237589626 || Train AUC: 0.6181251086109083 || Test AUC: 0.6133169172317288
epoch: 5 || train_loss: 51241.23891401896 || valid_loss: 17654.418393742206 || Train AUC: 0.6270589233604251 || Test AUC: 0.6235857296373006
epoch: 6 || train_loss: 51030.745090222044 || valid_loss: 17573.648879922534 || Train AUC: 0.6327921971323467 || Test AUC: 0.630652689443121
epoch: 7 || train_loss: 50867.28719779729 || valid_loss: 17518.333903495266 || Train AUC: 0.6368837718191298 || Test AUC: 0.635785

In [168]:
predict_proba

array([0.24788536, 0.26183463, 0.39297223, ..., 0.23319043, 0.19535697,
       0.1335829 ])

# predict_proba

In [169]:
a = list(predict_proba)

In [170]:
len(a)

59474

In [171]:
test.drop(['label'], axis=1, inplace=True)

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38787/3233372322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(['label'], axis=1, inplace=True)


In [172]:
test['predict_proba'] = a
test

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_38787/3906817452.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predict_proba'] = a


,year,country_id,item_id,GDP,preference,GDP_growth,population,pop_growth,USD,WTI,predict_proba
0,2013,0,1.0,0.000881,1.730195e-08,0.017864,0.022325,0.035276,1.00000,1.000000,0.247885
1,2013,0,2.0,0.000881,1.115475e-07,0.017864,0.022325,0.035276,1.00000,1.000000,0.261835
2,2013,0,3.0,0.000881,1.524764e-05,0.017864,0.022325,0.035276,1.00000,1.000000,0.392972
3,2013,0,4.0,0.000881,3.985107e-07,0.017864,0.022325,0.035276,1.00000,1.000000,0.241753
4,2013,0,5.0,0.000881,1.581240e-07,0.017864,0.022325,0.035276,1.00000,1.000000,0.234680
...,...,...,...,...,...,...,...,...,...,...,...
59469,2018,202,89.0,0.001464,1.069936e-05,0.942353,0.010650,0.020411,1.00558,0.439715,0.219267
59470,2018,202,91.0,0.001464,7.333262e-07,0.942353,0.010650,0.020411,1.00558,0.439715,0.218932
59471,2018,202,92.0,0.001464,6.773078e-07,0.942353,0.010650,0.020411,1.00558,0.439715,0.233190
59472,2018,202,97.0,0.001464,7.280952e-10,0.942353,0.010650,0.020411,1.00558,0.439715,0.195357


In [189]:
test.to_csv('../processed/FM_result.csv', index=False)

In [190]:
result_13_18 = test

# 성능평가

In [191]:
# precision 함수로 만들기
def FM_precision(country, current_year, new_exp_Nyears, proba_threshold):
    c_items = new_exp_Nyears[(idx_to_country[country], current_year)] # 원래 수출 없다가 향후 N년 내에 수출한 item
    rec_13_18 = result_13_18[result_13_18['predict_proba'] > proba_threshold]
    rec_items = rec_13_18[rec_13_18['country_id']==country].query('year==@current_year').item_id.tolist() # 추천 item
    hit_items = [i for i in c_items if i in rec_items]
    if len(rec_items) == 0:
        if len(c_items) == 0:
            precision = 1
        else:
            precision = 0
    else:
        precision = len(hit_items) / len(rec_items) # 추천한 item 중에서 몇개를 실제로 수출했는지
    return precision

# 해당년도 모든 나라에 대해서 precision 계산
def FM_precision_year(year, new_exp_Nyears, proba_threshold=0.5):
    precision = []
    c_len = len(idx_to_country)
    for country in range(0, c_len):
        precision.append(FM_precision(country, year, new_exp_Nyears, proba_threshold))
    return sum(precision) / c_len

# 전체 데이터에 대해서 precision 계산
def FM_precision_all(proba_threshold=0.5):
    precision = []
    c_len = len(idx_to_country) # 국가 수
    years = range(2013, 2019)
    for country in range(0, c_len):
        for year in years:
                precision.append(FM_precision(country, year, new_exp_3years, proba_threshold))
    return sum(precision) / (c_len*len(years))

In [192]:
# recall 함수로 만들기
def FM_recall(country, current_year, new_exp_Nyears, proba_threshold):
    c_items = new_exp_Nyears[(idx_to_country[country], current_year)]  # 원래 수출 없다가 향후 N년 내에 수출한 item
    rec_13_18 = result_13_18[result_13_18['predict_proba'] > proba_threshold]  
    rec_items = rec_13_18[rec_13_18['country_id']==country].query('year==@current_year').item_id.tolist() 
    hit_items = [i for i in c_items if i in rec_items] # 추천한 item 중에서 실제로 수출한 item
    if len(c_items) == 0:
        if len(rec_items) == 0:
            recall = 1
        else:
            recall = 0
    else:
        recall = len(hit_items) / len(c_items) # 전체 수출한 item 중에서 몇개를 추천했는지
    return recall

# 해당년도 모든 나라에 대해서 recall 계산
def FM_recall_year(year, new_exp_Nyears, proba_threshold=0.5):
    recall = []
    c_len = len(idx_to_country)
    cnt = 0
    for country in range(0, c_len):
        recall.append(FM_recall(country, year, new_exp_Nyears, proba_threshold))
    return sum(recall) / c_len

# 전체 데이터에 대해서 recall 계산
def FM_recall_all(proba_threshold=0.5):
    recall = []
    c_len = len(idx_to_country)  # 국가 수
    years = range(2013, 2019)
    for country in range(0, c_len):
        for year in years:
            recall.append(FM_recall(country, year, new_exp_3years, proba_threshold))

    return sum(recall) / (c_len*len(years))

In [193]:
def FM_f1_score(country, current_year, new_exp_Nyears, proba_threshold):
    precision = FM_precision(country, current_year, new_exp_Nyears, proba_threshold)
    recall = FM_recall(country, current_year, new_exp_Nyears, proba_threshold)
    if precision == 0 and recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def FM_f1_score_all(proba_threshold=0.5):
    f1_score = []
    c_len = len(idx_to_country)  # 국가 수
    years = range(2013, 2019)
    for country in range(0, c_len):
        for year in years:
            f1_score.append(FM_f1_score(country, year, new_exp_3years, proba_threshold))
    return sum(f1_score) / (c_len * len(years))

In [194]:
def get_best_combination(metric):
    best_score = 0
    best_thres = 0
    for thres in tqdm(np.arange(0.1,0.9, 0.01)):
        score = metric(thres)
        if score >= best_score:
            best_score = score
            best_thres = thres
    return best_score, best_thres

In [195]:
get_best_combination(FM_precision_all)

100%|██████████| 80/80 [01:42<00:00,  1.28s/it]


(0.4615124449078221, 0.38999999999999985)

In [197]:
get_best_combination(FM_recall_all)

100%|██████████| 80/80 [01:41<00:00,  1.27s/it]


(0.9810579404175462, 0.1)

In [198]:
get_best_combination(FM_f1_score_all)

100%|██████████| 80/80 [03:22<00:00,  2.53s/it]


(0.4561701329437273, 0.2899999999999999)

In [199]:
FM_precision_all(0.2899999999999999)

0.3847488443691743

# 직접 보기

In [216]:
country = 54
current_year = 2018
proba_threshold = 0.38999999999999985

print('pricision : ', FM_precision(country, current_year, new_exp_3years, proba_threshold))


print('국가 : ',idx_to_country[country])

#정답 리스트
c_items = new_exp_3years[(idx_to_country[country], current_year)]
print('정답 : ',c_items)


# 추천한 리스트
rec_13_18 = result_13_18[result_13_18['predict_proba'] > proba_threshold]  
rec_items = rec_13_18[rec_13_18['country_id']==country].query('year==@current_year').item_id.tolist()
print('추천 : ',rec_items)

pricision :  1.0
국가 :  Ecuador
정답 :  [1.0, 65.0, 9.0, 44.0, 13.0, 15.0, 16.0, 17.0, 47.0, 79.0, 20.0, 53.0, 31.0]
추천 :  [20.0]


In [222]:
item_name = pd.read_csv('../processed/item_name-code.csv')
item_name.query('품목코드 == 1.0')

,품목명,품목코드
0,살아 있는 동물,1.0


In [217]:
country = 72
current_year = 2018
proba_threshold = 0.38999999999999985

print('pricision : ', FM_precision(country, current_year, new_exp_3years, proba_threshold))


print('국가 : ',idx_to_country[country])

#정답 리스트
c_items = new_exp_3years[(idx_to_country[country], current_year)]
print('정답 : ',c_items)


# 추천한 리스트
rec_13_18 = result_13_18[result_13_18['predict_proba'] > proba_threshold]  
rec_items = rec_13_18[rec_13_18['country_id']==country].query('year==@current_year').item_id.tolist()
print('추천 : ',rec_items)

pricision :  0.14285714285714285
국가 :  Greenland
정답 :  [33.0, 85.0]
추천 :  [19.0, 27.0, 30.0, 33.0, 34.0, 38.0, 90.0]


In [227]:
i = country_to_idx['Thailand']
country = i
current_year = 2017
proba_threshold = 0.38999999999999985

print('pricision : ', FM_precision(country, current_year, new_exp_3years, proba_threshold))


print('국가 : ',idx_to_country[country])

#정답 리스트
c_items = new_exp_3years[(idx_to_country[country], current_year)]
print('정답 : ',c_items)


# 추천한 리스트
rec_13_18 = result_13_18[result_13_18['predict_proba'] > proba_threshold]  
rec_items = rec_13_18[rec_13_18['country_id']==country].query('year==@current_year').item_id.tolist()
print('추천 : ',rec_items)

pricision :  1.0
국가 :  Thailand
정답 :  [43.0, 46.0]
추천 :  [43.0]
